<a href="https://colab.research.google.com/github/xiner-hash/feature_engineering-/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
import pandas as pd
import numpy as np

from numpy import loadtxt

from sklearn import tree
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

import statsmodels.api as sm

from statistics import *

from xgboost import XGBClassifier

from matplotlib import pyplot

from sklearn import tree, metrics
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score


drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Time-related

In [144]:
"""
action = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')
affect = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')
submission = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/submission.csv',delimiter=';')
user = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/user.csv',delimiter=';')
user['pre_test'] = pd.to_numeric(user['pre_test'].str.replace(',', ''))
user['mid_test'] = pd.to_numeric(user['mid_test'].str.replace(',', ''))
"""

"\naction = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')\naffect = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')\nsubmission = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/submission.csv',delimiter=';')\nuser = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/user.csv',delimiter=';')\nuser['pre_test'] = pd.to_numeric(user['pre_test'].str.replace(',', ''))\nuser['mid_test'] = pd.to_numeric(user['mid_test'].str.replace(',', ''))\n"

In [145]:
df = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')

In [146]:
# convert action_date column to datetime format
df['action_date'] = pd.to_datetime(df['action_date'])


df['time_of_day'] = pd.cut(df['action_date'].dt.hour,
                           bins=[6, 12, 18, 24],
                           labels=['Morning', 'Afternoon', 'Evening'],
                           include_lowest=False,
                           ordered=False)


# generate Extreme Time of Day feature
df['extreme_time_of_day'] = pd.cut(df['action_date'].dt.hour,
                                   bins=[0, 4, 8, 21, 24],
                                   labels=['late_night', 'early_morning', 'non_extreme_hours', 'late_night'],
                                   ordered=False)

# generate Day of Week feature
df['day_of_week'] = df['action_date'].dt.day_name()
df['day_of_week'] = df['day_of_week'].apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday')


# Create dummy variables
time_of_day_dummies = pd.get_dummies(df['time_of_day'], prefix='time_of_day')
extreme_time_of_day_dummies = pd.get_dummies(df['extreme_time_of_day'], prefix='extreme_time_of_day')
day_of_week_dummies = pd.get_dummies(df['day_of_week'], prefix='day_of_week')

# concatenate dummy variables with original dataframe
df = pd.concat([df, time_of_day_dummies, extreme_time_of_day_dummies, day_of_week_dummies], axis=1)

In [147]:
df = df.drop(['time_of_day', 'extreme_time_of_day','day_of_week'], axis=1)


In [148]:
df = df.drop(["action_date","task_id"], axis=1)

In [149]:
df2 = df

## Percentage Correctness

In [150]:
# df = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')


In [151]:
df

,task_submit_id,user_id,test_id,test_succeeded,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend
0,207153,1582,1,1,0,0,1,0,0,1,1,0
1,207157,1582,1,1,0,0,1,0,0,1,1,0
2,207173,1573,1,1,0,0,0,1,0,0,1,0
3,207178,1573,1,0,0,0,0,1,0,0,1,0
4,207180,1573,1,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
75214,291783,1681,1,0,0,1,0,0,0,1,0,1
75215,291784,1681,1,1,0,1,0,0,0,1,0,1
75216,291786,1681,1,0,0,1,0,0,0,1,0,1
75217,291787,1681,1,1,0,1,0,0,0,1,0,1


In [152]:
# Create new columns for the desired features
df["last_3_attempts_incorrect"] = df.groupby("user_id")["test_succeeded"].rolling(3).sum().reset_index(level=0, drop=True)
df["last_5_attempts_incorrect"] = df.groupby("user_id")["test_succeeded"].rolling(5).sum().reset_index(level=0, drop=True)
df["last_8_attempts_incorrect"] = df.groupby("user_id")["test_succeeded"].rolling(8).sum().reset_index(level=0, drop=True)
df["last_3_attempts_correct"] = df.groupby("user_id")["test_succeeded"].rolling(3).apply(lambda x: x[::-1].cumsum()[::-1][0], raw=True).reset_index(level=0, drop=True)
df["last_5_attempts_correct"] = df.groupby("user_id")["test_succeeded"].rolling(5).apply(lambda x: x[::-1].cumsum()[::-1][0], raw=True).reset_index(level=0, drop=True)
df["last_8_attempts_correct"] = df.groupby("user_id")["test_succeeded"].rolling(8).apply(lambda x: x[::-1].cumsum()[::-1][0], raw=True).reset_index(level=0, drop=True)

# Replace NaN values with 0
df.fillna(0, inplace=True)

In [153]:
#df = pd.merge(df, df2, on='task_submit_id')

In [154]:
df

,task_submit_id,user_id,test_id,test_succeeded,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct
0,207153,1582,1,1,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0
1,207157,1582,1,1,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0
2,207173,1573,1,1,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0
3,207178,1573,1,0,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0
4,207180,1573,1,1,0,0,0,1,0,0,1,0,2.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75214,291783,1681,1,0,0,1,0,0,0,1,0,1,0.0,1.0,2.0,0.0,1.0,2.0
75215,291784,1681,1,1,0,1,0,0,0,1,0,1,1.0,2.0,3.0,1.0,2.0,3.0
75216,291786,1681,1,0,0,1,0,0,0,1,0,1,1.0,1.0,2.0,1.0,1.0,2.0
75217,291787,1681,1,1,0,1,0,0,0,1,0,1,2.0,2.0,3.0,2.0,2.0,3.0


## Attempt Count

In [155]:
# create new columns for the features
df['cumulative_correct'] = 0
df['cumulative_incorrect'] = 0

# initialize variables to keep track of correct and incorrect answers
correct_count = 0
incorrect_count = 0
current_task_id = None

# loop through the rows of the DataFrame
for i, row in df.iterrows():
    # if the user_id changed, reset the counts
    if row['user_id'] != current_task_id:
        correct_count = 0
        incorrect_count = 0
        current_task_id = row['user_id']
    
    # update the counts based on the test_succeeded
    if row['test_succeeded'] == 1:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # update the new columns in the DataFrame
    df.at[i, 'cumulative_correct'] = correct_count
    df.at[i, 'cumulative_incorrect'] = incorrect_count


In [156]:
df

,task_submit_id,user_id,test_id,test_succeeded,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct,cumulative_correct,cumulative_incorrect
0,207153,1582,1,1,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,207157,1582,1,1,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
2,207173,1573,1,1,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,207178,1573,1,0,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
4,207180,1573,1,1,0,0,0,1,0,0,1,0,2.0,0.0,0.0,2.0,0.0,0.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75214,291783,1681,1,0,0,1,0,0,0,1,0,1,0.0,1.0,2.0,0.0,1.0,2.0,11,19
75215,291784,1681,1,1,0,1,0,0,0,1,0,1,1.0,2.0,3.0,1.0,2.0,3.0,12,19
75216,291786,1681,1,0,0,1,0,0,0,1,0,1,1.0,1.0,2.0,1.0,1.0,2.0,12,20
75217,291787,1681,1,1,0,1,0,0,0,1,0,1,2.0,2.0,3.0,2.0,2.0,3.0,13,20


In [157]:
df = df.drop(["user_id","test_id",'test_succeeded'], axis=1)

## **Model**

In [158]:
df2 = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')

In [159]:
df2 = df2.drop(['task_id',"action_date"], axis=1)


In [160]:
df = pd.merge(df, df2, on='task_submit_id')
df = df.drop(['task_submit_id'], axis=1)


In [161]:
df

,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct,cumulative_correct,cumulative_incorrect,user_id,affect
0,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1582,AN
1,0,0,0,1,0,0,1,0,2.0,3.0,0.0,2.0,3.0,0.0,3,2,1573,FO
2,0,0,0,1,0,0,1,0,2.0,3.0,0.0,2.0,3.0,0.0,3,2,1573,OT
3,0,0,0,1,0,0,1,0,2.0,3.0,0.0,2.0,3.0,0.0,4,2,1573,FO
4,0,0,0,1,0,0,1,0,2.0,3.0,0.0,2.0,3.0,0.0,4,2,1573,OT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19259,0,0,1,0,0,1,1,0,0.0,1.0,1.0,0.0,1.0,1.0,1,4,1674,FO
19260,1,0,0,0,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1809,CO
19261,1,0,0,0,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,2,1809,CO
19262,1,0,0,0,0,1,0,1,1.0,1.0,1.0,1.0,1.0,1.0,1,2,1809,CO


In [164]:
unique_affect = df['affect'].unique()
print(unique_affect)

affect_counts = df['affect'].value_counts()
print(affect_counts)


['AN' 'FO' 'OT' 'BO' 'CO' 'FR']
FO    9863
FR    2807
OT    2217
AN    1978
CO    1520
BO     879
Name: affect, dtype: int64


In [165]:
features = df.loc[:, 'time_of_day_Morning':'cumulative_incorrect']

# SumattemptCount (no time) or time_of_day_Morning

features.fillna(features.mean(), inplace=True)
students = df['user_id'].values.tolist()

In [167]:
affect_labels = ['AN','FO','OT','BO','CO','FR']
affect_label = df['affect']

### **Random Forest**

In [170]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = RandomForestClassifier(n_estimators=100,random_state=1)

        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5023882575972389
Affective label:  FO
AUC ROC performance:  0.49508509385544724
Affective label:  OT
AUC ROC performance:  0.4962437160992583
Affective label:  BO
AUC ROC performance:  0.4984842487452969
Affective label:  CO
AUC ROC performance:  0.4965302943391962
Affective label:  FR
AUC ROC performance:  0.4934939932144524


### **ANN**

In [176]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = MLPClassifier(random_state=1,solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2, 3))


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Affective label:  FO
AUC ROC performance:  0.49263445283595714
Affective label:  OT
AUC ROC performance:  0.5
Affective label:  BO
AUC ROC performance:  0.4997313272434175
Affective label:  CO
AUC ROC performance:  0.5
Affective label:  FR
AUC ROC performance:  0.5


### **Decision Tree**

In [177]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = DecisionTreeClassifier(random_state=1,max_leaf_nodes= None, min_samples_leaf= 2, min_samples_split=10)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.4996266472323351
Affective label:  FO
AUC ROC performance:  0.49959856196770236
Affective label:  OT
AUC ROC performance:  0.4987252397941484
Affective label:  BO
AUC ROC performance:  0.4988472904468952
Affective label:  CO
AUC ROC performance:  0.49925442962943684
Affective label:  FR
AUC ROC performance:  0.4957557206314287


### **SVM**

In [173]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = svm.SVC(kernel='rbf', decision_function_shape='ovr',random_state = 1)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5
Affective label:  FO
AUC ROC performance:  0.5062871157270963
Affective label:  OT
AUC ROC performance:  0.49994186046511635
Affective label:  BO
AUC ROC performance:  0.5
Affective label:  CO
AUC ROC performance:  0.5
Affective label:  FR
AUC ROC performance:  0.4990399448496012


### **Naive Bayes**

In [174]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = GaussianNB(priors=[0.1, 0.9])


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))




Affective label:  AN
AUC ROC performance:  0.5241575617088999
Affective label:  FO
AUC ROC performance:  0.48994956031310766
Affective label:  OT
AUC ROC performance:  0.4868886756597803
Affective label:  BO
AUC ROC performance:  0.5098679711428037
Affective label:  CO
AUC ROC performance:  0.5105701790693656
Affective label:  FR
AUC ROC performance:  0.5031352636781135


### **XGB**

In [175]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = XGBClassifier(learning_rate=0.05, n_estimators=50, random_state=1)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.49931386863392957
Affective label:  FO
AUC ROC performance:  0.5026429413920507
Affective label:  OT
AUC ROC performance:  0.5003704013678245
Affective label:  BO
AUC ROC performance:  0.49989110546406906
Affective label:  CO
AUC ROC performance:  0.49974846814920476
Affective label:  FR
AUC ROC performance:  0.49786627654866394


In [ ]:
features